# Degree Day Analysis


In [16]:
# Import the required libraries and dependencies
import os
from dotenv import load_dotenv
from pathlib import Path
import datetime as dt
import pandas as pd
import hvplot.pandas
import plotly.express as px

In [4]:
import import_data as gd

## Enable your Mapbox API access token

In [5]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv("MAPBOX_API_ACCESS_TOKEN")

# Confirm the availability of your Mapbox API access token by checking its type
if type(mapbox_api_access_token) == type(''):
    # Set your Mapbox API access token
    px.set_mapbox_access_token(mapbox_api_access_token)
    print('Mapbox API loaded.')
else:
    print('Error loading Mapbox API Token. Check your .env file.')

Mapbox API loaded.


## Import the data

In [6]:
ercot_rtm_2020 = gd.get_ercot_year_df(gd.rtm_path_prefix, '2020')
ercot_rtm_2020.head()

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
0,01/01/2020,1.0,1.0,N,HB_BUSAVG,SH,12.46
1,01/01/2020,1.0,2.0,N,HB_BUSAVG,SH,14.22
2,01/01/2020,1.0,3.0,N,HB_BUSAVG,SH,14.06
3,01/01/2020,1.0,4.0,N,HB_BUSAVG,SH,14.09
4,01/01/2020,1.0,1.0,N,HB_HOUSTON,HU,12.46


In [7]:
ercot_rtm_2021 = gd.get_ercot_year_df(gd.rtm_path_prefix, '2021')
ercot_rtm_2021.head()

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
0,01/01/2021,1,1,N,HB_BUSAVG,SH,15.54
1,01/01/2021,1,2,N,HB_BUSAVG,SH,15.38
2,01/01/2021,1,3,N,HB_BUSAVG,SH,15.27
3,01/01/2021,1,4,N,HB_BUSAVG,SH,14.73
4,01/01/2021,1,1,N,HB_HOUSTON,HU,15.86


In [9]:
ercot_rtm_df = pd.concat(
    [ercot_rtm_2020, ercot_rtm_2021], 
    ignore_index=True,
)

In [10]:
display(ercot_rtm_df.head(10))
display(ercot_rtm_df.tail(10))

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
0,01/01/2020,1.0,1.0,N,HB_BUSAVG,SH,12.46
1,01/01/2020,1.0,2.0,N,HB_BUSAVG,SH,14.22
2,01/01/2020,1.0,3.0,N,HB_BUSAVG,SH,14.06
3,01/01/2020,1.0,4.0,N,HB_BUSAVG,SH,14.09
4,01/01/2020,1.0,1.0,N,HB_HOUSTON,HU,12.46
5,01/01/2020,1.0,2.0,N,HB_HOUSTON,HU,14.22
6,01/01/2020,1.0,3.0,N,HB_HOUSTON,HU,14.06
7,01/01/2020,1.0,4.0,N,HB_HOUSTON,HU,14.09
8,01/01/2020,1.0,1.0,N,HB_HUBAVG,AH,12.46
9,01/01/2020,1.0,2.0,N,HB_HUBAVG,AH,14.22


,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
1260668,07/24/2021,24,4,N,LZ_SOUTH,LZEW,25.70
1260669,07/24/2021,24,1,N,LZ_WEST,LZ,33.66
1260670,07/24/2021,24,1,N,LZ_WEST,LZEW,33.65
1260671,07/24/2021,24,2,N,LZ_WEST,LZ,36.15
1260672,07/24/2021,24,2,N,LZ_WEST,LZEW,36.15
1260673,07/24/2021,24,3,N,LZ_WEST,LZEW,30.08
1260674,07/24/2021,24,3,N,LZ_WEST,LZ,30.07
1260675,07/24/2021,24,4,N,LZ_WEST,LZEW,27.21
1260676,07/24/2021,24,4,N,LZ_WEST,LZ,27.21
1260677,2021-07-25 00:00:00,NaN,NaN,10,NaN,08:03:41,NaN


## Combine Delivery Date, Delivery Hour, and Deliver Interval into datetime

In [46]:
interval_minutes = [0, 15, 30, 45]
def create_datetime(df, row_index):
    date = df.iloc[row_index]['Delivery Date']
    hour = int(df.iloc[row_index]['Delivery Hour'])
    interval = int(df.iloc[row_index]['Delivery Interval'])

    format_str = '%m/%d/%Y' # The format
    datetime_obj = dt.datetime.strptime(date, format_str)
    time_delta = dt.timedelta(hours=hour, minutes=interval_minutes[interval-1])
    date_time = datetime_obj + time_delta
    return date_time
  
print(create_datetime(ercot_rtm_df, 0))

2020-01-01 01:00:00


In [47]:
for i in ercot_rtm_df.index:
    ercot_rtm_df['Datetime'] = create_datetime(ercot_rtm_df, i)
#    print(f'\rUpdating datetime: {i}', end='')
#print()

KeyboardInterrupt: 

In [ ]:
display(ercot_rtm_df.head())